In [ ]:
# @title
from IPython.display import HTML

# HTML y CSS para el título y la imagen en Google Colab
html_code = """
<style>
  #container {
    display: flex;
    justify-content: space-between;
    align-items: center;
    padding: 10px;
  }

  #title {
    font-size: 50px;
    font-weight: bold;
    color: black;
  }

  #logo {
    width: 174px;
    height: 101.5px;
  }
</style>

<div id="container">
  <div id="title">SunScan</div>
  <img id="logo" src="https://www.valoraanalitik.com/wp-content/uploads/2023/10/bid-696x406.jpg" alt="Logo">
</div>
"""

# Mostrar el código HTML en Colab
display(HTML(html_code))


In [ ]:
#@title 1. Configurar ambiente
%%time
import warnings, os

# Desactivar todos los warnings
warnings.filterwarnings("ignore")

!git clone https://github.com/EL-BID/Sunscan.git
%cd '/content/Sunscan'
!git config --global advice.detachedHead false
!git checkout e9f5ffe

!pip install --user -r requirements.txt

# Reiniciar el entorno de ejecución de Google Colab
os.kill(os.getpid(), 9)

In [ ]:
#@title 2. Cargar dependencias

import warnings
warnings.filterwarnings("ignore")

import os,glob, sys

fol_sys='/content/Sunscan/scripts'
if fol_sys not in sys.path:
    sys.path.append(fol_sys)

from samgeo.text_sam import LangSAM
import leafmap
import city_from_coord as cfc
import ultimo_archivo as ua
import sunlight_hours as sunh
import solar_energy_shp as ses
import datetime

sam = LangSAM()

In [ ]:
#@title 3. Escoger un área sobre el mapa para realizar el análisis

m_bb=leafmap.Map()
m_bb.add_basemap("Google Satellite")
m_bb

In [ ]:
#@title 4. Obtener la imagen


bb=m_bb.user_roi_bounds()
north, south, east, west = bb[3], bb[1], bb[2], bb[0]

lat,lon=((north+south)/2),((east+west)/2)#Centroid
str_bb=str(bb[3])+'_'+str(bb[1])+'_'+str(bb[2])+'_'+str(bb[0])

#Get name of the city with coordinates
city=cfc.city_from_coord(latitude=lat, longitude=lon)
city_name=str(city).replace("'",'').replace('(','').replace(')','').replace(', ','_').replace(' ','').replace('/','')
city_name=city_name+'_'+str_bb

results_fol='/content/results'
#Crear la carpeta donde estarán las imágenes y predicciones
if not os.path.exists(results_fol):
        os.makedirs(results_fol)

z=20
#image = '/content/'+city_name+'_z{}.tif'.format(str(z))
image=os.path.join(results_fol,city_name+'_{}z.tif'.format(str(z)))
leafmap.tms_to_geotiff(image, bb, zoom=z,
                       source='Satellite')

In [ ]:
#@title 5. Visualizar la imagen

#path='/content'
palabra_clave='z.tif'
#img_in=ua.ultimo_archivo('/content', '.tif')
img_in=ua.ultimo_archivo(results_fol, '.tif')

m0=leafmap.Map()
m0.add_raster(img_in)
#m0.add_vector(ua.ultimo_archivo('/content', '.shp'))
m0

In [ ]:
#@title 6. Usar SAM (Segment Anything by Meta) para detectar y segmentar los tejados

text_prompt='house'
sam.predict(img_in, text_prompt, box_threshold=0.24, text_threshold=0.24)

#Crear el raster
tif_out=img_in.replace('.tif','_{}.tif'.format(text_prompt))
#tif_out=img_in.replace('.tif','_sam_rooftop.tif')
sam.show_anns(  cmap='Greys_r',  add_boxes=False,  alpha=1,
  #title='Segmentation of {}'.format(text_prompt),
  title='Segmentación  de tejados usando SAM', blend=False,  output=tif_out)

#Crear el shp
shp_out=tif_out.replace('.tif','.shp')
sam.raster_to_vector(tif_out,shp_out)

m1=leafmap.Map()
m1.add_raster(img_in)
m1.add_vector(shp_out)
m1

In [ ]:
#@title 7. Energía solar: Seleccionar parámetros

from IPython.display import display, HTML
import ipywidgets as widgets

# Inyectar CSS personalizado para las descripciones de los widgets
display(HTML("""
<style>
    .widget-label {
        font-weight: bold !important;
        font-size: 16px !important;
        color: black !important;
    }
</style>
"""))

texto = 'Seleccionar los parámetros en las siguientes barras'

display(HTML(f'<p style="font-weight: bold; font-size: 20px;">{texto}</p>'))

# Configuración de estilo y layout personalizado para los sliders
custom_style = {'handle_color': 'white', 'description_width': 'initial'}
custom_layout = widgets.Layout(width='60%', height='80px', background_color='black', border_radius='10px', border_color='black')

metros_cuadrados_widget = widgets.IntSlider(
    value=4, min=2, max=10, step=1, description='Tamaño del panel solar (m2)',
    style=custom_style, layout=custom_layout)

potencia_widget = widgets.IntSlider(
    value=400, min=200, max=1000, step=100, description='Potencia de los páneles (watts)',
    style=custom_style, layout=custom_layout)

disponib_widget = widgets.IntSlider(
    value=50, min=0, max=100, step=10, description='Área disponible para páneles (%)',
    style=custom_style, layout=custom_layout)

vertical_box = widgets.VBox([metros_cuadrados_widget, potencia_widget, disponib_widget])

display(vertical_box)


texto2 = '<b>Nota:</b> Ejecutar la celda antes de seleccionar los parámetros'

display(HTML(f'<p style="font-size: 16px;">{texto2}</p>'))

In [ ]:
#@title 8. Energía solar: Estimación de generación anual

#Parámetros
panel_size=metros_cuadrados_widget.value
area_disp=disponib_widget.value/100
factor_dim=0.8
panel_pot=potencia_widget.value/1000

palabra_clave='.shp'
shp_in=ua.ultimo_archivo(results_fol, palabra_clave)
shp_out=shp_in.replace('.shp','_{}ps_{}ad_{}fd_{}pp.shp'.format(panel_size,area_disp,factor_dim,panel_pot))
excel_out=shp_in.replace('.shp','.xlsx')

start = datetime.datetime(2024, 1, 1, 0, 0, 0)
end = datetime.datetime(2024, 12, 31, 0, 0, 0)

delta = end - start

ses.solar_energy_shp(shp_in,shp_out,start,end,panel_size,area_disp,factor_dim,panel_pot,excel_out)

msol=leafmap.Map()
msol.add_raster(img_in)
msol.add_vector(shp_out)
msol

In [ ]:
#@title 9. Exportar el resultado (zip con todos los resultados)

import os,glob
from zipfile import ZipFile
from google.colab import files

# Ruta de la carpeta donde se buscarán los archivos. Ajusta según sea necesario.
carpeta = '/content/results'  # Ejemplo: './mi_carpeta_de_datos'

# Extensiones de archivos a incluir
extensiones = ['xlsx', 'shp', 'cpg', 'dbf', 'prj', 'shx']

# Crear una lista para todos los archivos a comprimir
archivos_a_comprimir = []
for extension in extensiones:
    archivos_a_comprimir.extend(glob.glob(f'{carpeta}/**/*.{extension}', recursive=True))

# Filtrar los archivos más recientes por extensión
archivos_recientes = {ext: max((f for f in archivos_a_comprimir if f.endswith('.' + ext)), key=os.path.getmtime) for ext in extensiones if any(f.endswith('.' + ext) for f in archivos_a_comprimir)}

# Nombre del archivo ZIP de salida
nombre_zip = 'SunScanResults.zip'

# Crear el archivo ZIP
with ZipFile(nombre_zip, 'w') as zipf:
    for archivo in archivos_recientes.values():
        zipf.write(archivo, os.path.basename(archivo))

# Descargar el archivo ZIP
files.download(nombre_zip)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#@title 10. Dashboard

import pandas as pd

list_excel = sorted(glob.glob('/content/results/*.xlsx'))

df=pd.DataFrame()

for c in list_excel:
  data=pd.read_excel(c)
  df=df.append(data)

df=df[df['area']>30]
df['Energia_anual_kw']=df['Energia_anual']/1000

import warnings

# Filtrar los warnings específicos que quieres evitar
warnings.filterwarnings("ignore", message=".*_PyDrive2ImportHook.*")
warnings.filterwarnings("ignore", message=".*_PyDriveImportHook.*")
warnings.filterwarnings("ignore", message=".*_GenerativeAIImportHook.*")
warnings.filterwarnings("ignore", message=".*_OpenCVImportHook.*")
warnings.filterwarnings("ignore", message=".*APICoreClientInfoImportHook.*")
warnings.filterwarnings("ignore", message=".*_BokehImportHook.*")
warnings.filterwarnings("ignore", message=".*_AltairImportHook.*")
warnings.filterwarnings("ignore", message=".*np.find_common_type.*")

#DASHBOARD
from IPython.display import HTML
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

num_edificaciones=df['Energia_anual'].count()
suma_energia_anual_kw=df['Energia_anual'].sum()/1000
suma_area=df['area'].sum()
prom_area=df['area'].mean()
prom_energia_anual_kw=suma_energia_anual_kw/num_edificaciones

# Función para formatear números con separadores de miles
def format_number(num):
    return "{:,.0f}".format(num)


html_code0 = """
<style>
  #container {
    display: flex;
    justify-content: space-between;
    align-items: center;
    padding: 10px;
  }

  #title {
    font-size: 25px;
    font-weight: bold;
    color: black;
  }

  #logo {
    width: 130.5px;
    height: 83.25px;
  }
</style>

<div id="container">
  <div id="title">SunScan Summary Results</div>
  <img id="logo" src="https://www.valoraanalitik.com/wp-content/uploads/2023/10/bid-696x406.jpg" alt="Logo">
</div>
"""

# Mostrar el código HTML en Colab
display(HTML(html_code0))





# Generación del código HTML
html_code = f'''
<html>
<head>
    <style>
        .card-container {{
            display: flex;
            justify-content: space-around;
        }}
        .card {{
            background-color: black;
            color: yellow;
            font-weight: bold;
            font-size: 18px;
            margin: 10px;
            padding: 20px;
            border-radius: 10px;
            display: inline-block;
        }}
    </style>
</head>
<body>
    <div class="card-container">
        <div class="card">Tejados: {format_number(num_edificaciones)}</div>
        <div class="card">Área total: {format_number(suma_area)} m2</div>
        <div class="card">Energía generada año: {format_number(suma_energia_anual_kw)} KWh</div>
    </div>
</body>
</html>
'''



# Imprimir o guardar el HTML en un archivo
display(HTML(html_code))



#Mapa y tabla
data_sorted = df.sort_values(by='Energia_anual', ascending=False)

# Crear textos personalizados para cada punto en el mapa
textos = ["<b>Área:</b> " + str(area) + "<br><b>Energía anual:</b> " + str(energia)
          for area, energia in zip(data_sorted['area'].round(0), data_sorted['Energia_anual'].round(0))]

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scattermapbox"}, {"type": "table"}]], column_widths=[0.5, 0.5])

# Añadir el scatter mapbox al subplot con textos modificados
fig.add_trace(go.Scattermapbox(
    lat=data_sorted['latitude'],
    lon=data_sorted['longitude'],
    mode='markers+text',
    marker=go.scattermapbox.Marker(
        size=data_sorted['Energia_anual']/data_sorted['Energia_anual'].max()*30, # Tamaño relativo a Energia_anual
        color=data_sorted['Energia_anual'],
        colorscale='Viridis', # Escala de colores basada en Energia_anual
        colorbar=dict(x=-0.1),  # Posiciona la barra de colores a la izquierda del mapa
        showscale=True
    ),
    text=textos  # Usar la lista de textos personalizados aquí
    ),
    row=1, col=1
)

# Añadir la tabla go.Table al subplot
fig.add_trace(go.Table(
    header=dict(
        values=["Tejado", "Área tejado (m2)", "Energía generada año (KWh)"],
        align='left'
    ),
    cells=dict(
        values=[
            data_sorted.index + 1,  # Número de tejado, ajustado para iniciar en 1
            data_sorted['area'].round(0),  # Área tejado (m2)
            (data_sorted['Energia_anual']/1000).round(1)  # Energía anual, asegúrate de que el nombre de columna es correcto aquí
        ],
        align='left'
    )
), row=1, col=2)

# Ajustes finales del layout, específicamente para el mapa
fig.update_layout(
    mapbox=dict(
        style="carto-positron",
        zoom=17,
        center=dict(lat=data_sorted['latitude'].mean(), lon=data_sorted['longitude'].mean())  # Centrar el mapa
    ),
    mapbox_style="carto-positron",
    height=600
)

fig.show()